In [1]:
from social_activity_recognition import load_dataset
import tensorflow as tf
import numpy as np

In [2]:
DS = load_dataset('./skeleton_only')

In [3]:
data_size = DS[(DS['activity']==1) & (DS['session']==1)]['user1'].values[0].shape
data_points = np.concatenate([DS[(DS['activity']==1) & (DS['session']==1)]['user1'].values[0],np.ones((data_size[0],1,15))],axis=1)
#print(data_points)
random_transf = np.concatenate([np.concatenate([np.eye(3),np.zeros((1,3))],axis=0),np.array([2,3,4,1])[:,np.newaxis]],axis=1) #np.random.normal(size=(3,3))
#random_transf = [np.concatenate([np.eye(3),np.zeros((1,3))],axis=0).shape,np.array([2,3,4,1])[:,np.newaxis].shape]
random_transf
#random_transf
print(np.matmul(random_transf,data_points[0])[0:4,:].T)

[[4.33910258 5.94124143 6.44866218 1.        ]
 [2.19935787 2.6332197  4.19648699 1.        ]
 [2.67482167 4.48465942 4.41553324 1.        ]
 [4.94199166 5.09126741 3.80768768 1.        ]
 [1.62545096 3.269066   2.42920742 1.        ]
 [3.47739974 3.55563319 1.05072333 1.        ]
 [4.2699648  5.734872   6.44866117 1.        ]
 [2.2327765  4.13409646 4.19648716 1.        ]
 [2.48558321 3.45361392 4.14062077 1.        ]
 [4.94199166 4.97634437 5.36950197 1.        ]
 [1.62545096 3.31856771 2.42922204 1.        ]
 [3.47739974 3.28754654 5.77209436 1.        ]
 [4.19996664 1.48222492 6.0442382  1.        ]
 [2.26544406 4.28438812 4.31552155 1.        ]
 [2.29009699 5.42599847 4.12821733 1.        ]]


In [4]:
data_points.shape


(1793, 4, 15)

In [9]:
def make_transformation_layer(output_point_series, subname=''):
    
    Transform = tf.Variable(initial_value=tf.truncated_normal(shape=(output_point_series,3,3),stddev=1e-3,dtype=tf.float32),trainable=True,dtype=tf.float32)
    Translation = tf.Variable(initial_value=tf.truncated_normal(shape=(output_point_series,3,1),stddev=1e-2,dtype=tf.float32),trainable=True,dtype=tf.float32)
    Last_row = tf.Variable(initial_value=np.concatenate([np.zeros((output_point_series,1,3)),np.ones(((output_point_series,1,1)))],axis=2),
                           trainable=False,dtype=tf.float32)
    return tf.concat([tf.concat([Transform, Translation],axis=2),Last_row],axis=1,name='Transform_matrix' + subname)
    

In [10]:
n_classes = 8
ind_channels = [50,100]
scene_channels = [30]
fc_sizes = [100]


graph = tf.Graph()

with graph.as_default() as g:
    
    
    
    user1_ph = tf.placeholder(shape=(None,1,4,15),dtype=tf.float32)
    user2_ph = tf.placeholder(shape=(None,1,4,15),dtype=tf.float32)
    
    label_ph = tf.placeholder(shape=(None,1,4,15),dtype=tf.float32)
    
    individual_transform1 = make_transformation_layer(output_point_series=ind_channels[0], subname='individual_1')
    individual_transform2 = make_transformation_layer(output_point_series=ind_channels[1], subname='individual_2')
    scene_transform = make_transformation_layer(output_point_series=scene_channels[0], subname='scene')
    
    transformed1_user1 = tf.matmul(individual_transform1, user1_ph, name='transformed1_user1')
    transformed1_user2 = tf.matmul(individual_transform1, user2_ph, name='transformed1_user2')
    
    transformed2_user1 = tf.matmul(individual_transform1, transformed1_user1, name='transformed2_user1')
    transformed2_user2 = tf.matmul(individual_transform1, transformed1_user2, name='transformed2_user2')
    
    scene_pts = tf.concat([transformed2_user1,transformed2_user2],axis=3)
    
    transformed_scene = tf.matmul(scene_transform, scene_pts, name='transformed_scene')
    
    flattened_points = tf.layers.flatten(transformed_scene, name="flattened_points")
    
    fc_out_1 = tf.contrib.layers.fully_connected(
        inputs=flattened_points,
        num_outputs=fc_sizes[0],
        activation_fn=tf.nn.relu(),
        trainable=True
    )
    
    fc_final = tf.contrib.layers.fully_connected(
        inputs=fc_out_1,
        num_outputs=n_classes,
        activation_fn=tf.nn.relu(),
        trainable=True
    )
    
    prediction = tf.nn.softmax(logits=fc_final)
    
    
    loss = tf.nn.softmax_cross_entropy_with_logits_v2(
        labels=tf.one_hot(label_ph, depth=n_classes),
        logits=fc_final)
    optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
    
    train_step = optimizer.minimize(tf.reduce_mean(loss))
    
    

ValueError: Shapes must be equal rank, but are 1 and 2 for 'transformed1_user1' (op: 'BatchMatMul') with input shapes: [50,4,4], [?,1,4,15].

In [6]:
min_max_per_session=[]
for s in range(1,len(DS['session'].unique())+1):
    temp1 = []
    temp2 = []
    for a in range(1,len(DS['activity'].unique())+1):
        mask = (DS['activity']==a) & (DS['session']!=s)
        #print(s,a,DS[mask]['user1'].values[0].shape,DS[mask]['user2'].values[0].shape)
        v_cat = np.concatenate([DS[mask]['user1'].values[0], DS[mask]['user2'].values[0]],axis=0)
        temp1 += [v_cat.max(axis=2).max(axis=0)]
        temp2 += [v_cat.min(axis=2).min(axis=0)]
        #print(DS[mask]['user1'].values)
        
    min_max_per_session += [(np.concatenate(temp1).min(axis=0),np.concatenate(temp2).max(axis=0))]
    
#min_max_per_session
        
        
        

In [ ]:
for s in range(1,len(DS['session'].unique())+1):
    for a in range(1,len(DS['activity'].unique())+1):
        mask = (DS['activity']==a) & (DS['session']!=s)
        